# Analyser les ventes d'un site de e-commerce spécialisé dans les livres

L'entreprise pour qui vous travaillez souhaite analyser les ventes de son site et mieux connaitre ses clients, ainsi que le détail des ventes de ses produits

Vous pourrez télécharger l'ensemble des données [via ce lien](https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-analyst/dataset_P4.zip)  Les données que vous avez à disposition :   
* les ventes (appelées “Transactions”)  
* la liste des clients  
* la liste des produits.

## Nettoyage des données

Avant de pouvoir entrer dans le vif du sujet, il vous faudra faire un peu de nettoyage ! Par exemple, vous devrez faire des choix quant au traitement des valeurs manquantes et des valeurs aberrantes.

Pour vous guider, voici quelques choses à explorer :  
* l'évolution du Chiffre d'Affaire de manière mensuelle
* Tester les différentes jointures possibles entre les tables (pour rappel, il en existe [4 types](http://python-simple.com/python-pandas/concatenations-joins-dataframe.php ))
* Des valeurs abérrantes sont présentes dans la table *transaction*, identifiez-les et supprimez-les de votre étude.
* Regarder le nombre de transaction par client ou le nombre de CA par client

Vous aurez donc des choix à faire : imputation de valeurs manquantes, gestions de valeurs atypiques etc.

A noter : enregistrer vos données nettoyées dans un fichier csv, nous le réutiliserons ensuite pour étudier les corrélations